<a href="https://colab.research.google.com/github/nantmoe-theingi/airbnb-nz-deception-sentiment/blob/main/notebooks/06_run_airbnb_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
PROJECT_DRIVE_DIR = "/content/drive/MyDrive/Colab Notebooks/airbnb_nz_deception_sentiment"

from google.colab import drive
drive.mount('/content/drive')

import os
os.makedirs(PROJECT_DRIVE_DIR, exist_ok=True)
print("Drive project folder:", PROJECT_DRIVE_DIR)

Mounted at /content/drive
Drive project folder: /content/drive/MyDrive/Colab Notebooks/airbnb_nz_deception_sentiment


In [3]:
cd {PROJECT_DRIVE_DIR}

/content/drive/MyDrive/Colab Notebooks/airbnb_nz_deception_sentiment


In [ ]:
# Create a new .py file and open it for editing
file_name = "run_airbnb_inference.py"
with open(file_name, "w") as f:
    f.write("")  # empty file created
print(f"{file_name} created.")

run_airbnb_inference.py created.


In [3]:
!python run_airbnb_inference.py \
  --input_parquet data/airbnb_reviews_cleaned.parquet \
  --output_path outputs/airbnb_predictions.parquet \
  --deception_model_dir artifacts/distilbert_deception_sweep/best_model \
  --sentiment_model_dir artifacts/distilbert_sentiment_sweep/best_model

2025-10-28 22:21:36.786905: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761690096.806776    2189 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761690096.812684    2189 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761690096.827735    2189 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761690096.827773    2189 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761690096.827777    2189 computation_placer.cc:177] computation placer alr

In [4]:
!python run_airbnb_inference.py \
  --input_parquet data/airbnb_reviews_cleaned.parquet \
  --output_path outputs/airbnb_predictions_partition/ \
  --partition_by year month \
  --deception_model_dir artifacts/distilbert_deception_sweep/best_model \
  --sentiment_model_dir artifacts/distilbert_sentiment_sweep/best_model

2025-10-28 23:18:29.928352: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761693509.948310   16327 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761693509.954317   16327 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761693509.970534   16327 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761693509.970559   16327 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761693509.970563   16327 computation_placer.cc:177] computation placer alr

In [3]:
import pandas as pd

df = pd.read_parquet("outputs/airbnb_predictions.parquet")
print(df.shape)
print(df.columns.tolist()[-12:])   # show prediction columns
print(df[["sent_pred", "deception_pred"]].value_counts().head())

(1533422, 27)
['deception_logit_0', 'deception_logit_1', 'sent_logit_0', 'sent_logit_1', 'sent_logit_2', 'deception_prob_label_0', 'deception_prob_label_1', 'sent_prob_negative', 'sent_prob_neutral', 'sent_prob_positive', 'deception_pred', 'sent_pred']
sent_pred  deception_pred
2          0                 933980
           1                 532944
1          0                  38452
0          0                  17971
1          1                   5508
Name: count, dtype: int64


In [6]:
print(df.columns.tolist())

['review_id', 'listing_id', 'date', 'text', 'price_num', 'property_type', 'room_type', 'host_is_superhost', 'neighbourhood', 'region_name', 'region_parent_name', 'region_parent_parent_name', 'region', 'year', 'month', 'deception_logit_0', 'deception_logit_1', 'sent_logit_0', 'sent_logit_1', 'sent_logit_2', 'deception_prob_label_0', 'deception_prob_label_1', 'sent_prob_negative', 'sent_prob_neutral', 'sent_prob_positive', 'deception_pred', 'sent_pred']


In [7]:
import pandas as pd
import numpy as np

# === Load your datasets ===
reviews = pd.read_csv("data/airbnb_nz_reviews.csv")
listings = pd.read_csv("data/airbnb_nz_listings.csv")
preds = pd.read_parquet("outputs/airbnb_predictions.parquet")

RUNTIME_SECONDS = 3044.8  # from your log

# --- 1) Canonicalise keys/dates exactly for the columns you specified ---
def canon_id(s: pd.Series) -> pd.Series:
    return s.astype("string").str.strip().str.replace(r"\.0$", "", regex=True)

for df, cols in [
    (reviews,  ["id", "reviewer_id"]),
    (preds,    ["review_id", "listing_id", "date"]),
    (listings, ["id"]),
]:
    for c in cols:
        if c in df.columns:
            if c == "date":
                df[c] = pd.to_datetime(df[c], errors="coerce")
            else:
                df[c] = canon_id(df[c])

# --- 2) Use PREDICTIONS as the anchor ---
# Join preds -> reviews ONLY to fetch reviewer_id for predicted rows
preds_with_reviewer = preds.merge(
    reviews[["id", "reviewer_id"]],
    how="left",
    left_on="review_id",
    right_on="id"
)

# (optional) quick sanity checks
predictions_generated = len(preds)  # count of rows that actually have predictions
missing_reviewer_ids = preds_with_reviewer["reviewer_id"].isna().sum()



# --- 3) Compute metrics restricted to the prediction subset ---
total_reviews = int("1533422")  # full cleaned corpus (for coverage denominator)
coverage = predictions_generated / total_reviews if total_reviews else np.nan

# Unique listings seen in predictions
unique_listings_pred = preds["listing_id"].nunique()

# Unique reviewers *whose reviews appear in preds*
unique_reviewers_pred = preds_with_reviewer["reviewer_id"].nunique()

# Date span derived from preds['date'] (NOT from all reviews)
if "date" in preds.columns and preds["date"].notna().any():
    date_lo, date_hi = preds["date"].min(), preds["date"].max()
    date_span_txt = f"{date_lo:%b~%Y}--{date_hi:%b~%Y}"
else:
    date_span_txt = "N/A"

throughput = predictions_generated / RUNTIME_SECONDS if RUNTIME_SECONDS else np.nan

# --- 4) Build summary (values reflect prediction-anchored counts where applicable) ---
def fmt_int(n): return f"{int(n):,}".replace(",", "{,}")
def fmt_pct(x): return f"{x*100:.2f}\\%"

summary_rows = [
    ("Total reviews processed", "Reviews in the cleaned Airbnb corpus", fmt_int(total_reviews)),
    ("Predictions generated",   "Reviews with valid DistilBERT outputs", fmt_int(predictions_generated)),
    ("Prediction coverage",     "Share of total reviews with predictions", fmt_pct(coverage) if pd.notna(coverage) else "N/A"),
    ("Date span",               "Review period covered by predictions", date_span_txt),
    ("Unique listings",         "Distinct listings represented in predictions", fmt_int(unique_listings_pred)),
    ("Unique reviewers",        "Distinct reviewers represented in predictions", fmt_int(unique_reviewers_pred)),
    ("Runtime (s)",             "Total inference execution time", f"{RUNTIME_SECONDS:,.1f}".replace(",", "{,}")),
    ("Processing throughput",   "Reviews processed per second", f"{throughput:.0f}" if pd.notna(throughput) else "(insert value)"),
]

df_summary = pd.DataFrame(summary_rows, columns=["Metric", "Description", "Value"])

# --- 5) Save CSV ---
csv_path = "figures/airbnb_inference_integrity_summary.csv"
df_summary.to_csv(csv_path, index=False)
print(f"Saved: {csv_path}")
print(f"Missing reviewer_id after join: {missing_reviewer_ids:,}")


Saved: figures/airbnb_inference_integrity_summary.csv
Missing reviewer_id after join: 0


In [8]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

# --- Inputs ---
# preds = pd.read_parquet("outputs/airbnb_predictions.parquet")
# Must contain:
#   'deception_prob_label_1', 'sent_prob_negative', 'sent_prob_neutral',
#   'sent_prob_positive', 'date'

OUTDIR = "figures"
os.makedirs(OUTDIR, exist_ok=True)

# ============== Figure 1: Deception probability histogram ==============

def plot_deception_histogram(df: pd.DataFrame,
                             prob_col: str = "deception_prob_label_1",
                             out_path: str = f"{OUTDIR}/deception_probability_hist.png"):
    s = pd.to_numeric(df[prob_col], errors="coerce").dropna()
    plt.figure(figsize=(8, 5))
    # KDE-free histogram; fixed binning on [0,1]
    plt.hist(s, bins=50, range=(0, 1))
    plt.title("Distribution of Deception Probability")
    plt.xlabel("P(deceptive)")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.savefig(out_path, dpi=300)
    plt.close()
    return out_path

dec_hist_path = plot_deception_histogram(preds)

# ============== Figure 2: Monthly stacked area of mean sentiment probs ==============

def plot_monthly_sentiment_stack(df: pd.DataFrame,
                                 date_col: str = "date",
                                 neg_col: str = "sent_prob_negative",
                                 neu_col: str = "sent_prob_neutral",
                                 pos_col: str = "sent_prob_positive",
                                 out_csv: str = f"{OUTDIR}/monthly_sentiment_means.csv",
                                 out_png: str = f"{OUTDIR}/sentiment_composition_stacked_area.png"):
    z = df.copy()
    z[date_col] = pd.to_datetime(z[date_col], errors="coerce")
    z = z.dropna(subset=[date_col])

    # Monthly mean of probabilities
    monthly = (
        z.set_index(date_col)[[neg_col, neu_col, pos_col]]
         .resample("MS").mean()
    )

    # (Optional) Re-normalize each month to sum to 1 (protect against rounding)
    monthly = monthly.clip(lower=0)
    row_sums = monthly.sum(axis=1)
    monthly = monthly.div(row_sums.where(row_sums > 0, np.nan), axis=0)

    # Save CSV for reproducibility
    monthly_out = monthly.copy()
    monthly_out.index.name = "month"
    monthly_out.to_csv(out_csv)

    # Stacked area plot (0–100%)
    x = monthly.index
    neg = monthly[neg_col].values
    neu = monthly[neu_col].values
    pos = monthly[pos_col].values

    plt.figure(figsize=(9, 5.2))
    plt.stackplot(x, neg, neu, pos, labels=["Negative", "Neutral", "Positive"])
    plt.gca().yaxis.set_major_formatter(PercentFormatter(1.0))
    plt.ylim(0, 1)
    plt.title("Sentiment Probability Composition (Monthly Means)")
    plt.xlabel("Month")
    plt.ylabel("Share")
    plt.legend(loc="upper right", frameon=False)
    plt.tight_layout()
    plt.savefig(out_png, dpi=300)
    plt.close()

    return out_csv, out_png

sent_csv_path, sent_stack_path = plot_monthly_sentiment_stack(preds)

print("Saved:", dec_hist_path)
print("Saved:", sent_stack_path, "and CSV:", sent_csv_path)

Saved: figures/deception_probability_hist.png
Saved: figures/sentiment_composition_stacked_area.png and CSV: figures/monthly_sentiment_means.csv


In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import os

OUTDIR = "figures"
os.makedirs(OUTDIR, exist_ok=True)

# --- assume preds is your inference dataframe ---
# preds['deception_pred'] : 0 = truthful, 1 = deceptive

# 1️⃣ Count and normalize
counts = preds["deception_pred"].value_counts().sort_index()
share = counts / counts.sum() * 100
labels = ["truthful", "deceptive"]

# 2️⃣ Create bar chart
plt.figure(figsize=(6.5, 4.5))
bars = plt.bar(labels, share, color="#2E6EBA")

# 3️⃣ Add text labels on top of each bar
for bar, pct in zip(bars, share):
    plt.text(
        bar.get_x() + bar.get_width()/2,
        bar.get_height() + 1,
        f"{pct:.1f}%",
        ha="center",
        va="bottom",
        fontsize=11,
        fontweight="semibold"
    )

# 4️⃣ Customize plot style
plt.title("Distribution of Predicted Deception — Airbnb NZ (All Reviews)", fontsize=12)
plt.ylabel("Share of Reviews (%)")
plt.ylim(0, 105)
plt.tight_layout()

# 5️⃣ Save figure
out_path = f"{OUTDIR}/deception_predicted_bar.png"
plt.savefig(out_path, dpi=300)
plt.close()

print(f"Saved bar chart to {out_path}")


Saved bar chart to figures/deception_predicted_bar.png


In [14]:
print(listings.columns.tolist())

['id', 'listing_url', 'scrape_id', 'last_searched', 'last_scraped', 'source', 'name', 'description', 'neighborhood_overview', 'picture_url', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 'calendar_last_scraped', 'numbe

In [ ]:
unique_listings = merged_full["id"].nunique()
unique_reviewers = merged_full["listing_id"].nunique()

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# reviews_df: must contain at least ['id','listing_id','date','comments', ...]
#             and ideally prediction columns if you wrote them back to this frame
# listings_df: must contain at least ['id'] for unique listing count (optional but preferred)
# runtime_seconds: total wall time your inference pass took (float or int). If unknown, set None.

# Example:
reviews_df = pd.read_parquet("outputs/airbnb_predictions.parquet")
listings_df = pd.read_csv("data/airbnb_nz_listings.csv")
runtime_seconds = None

# --- Config (robust column picks) ---
PRED_COLS = {
    "deception_pred": ["deception_pred"],
    "sent_pred": ["sent_pred"]
}
REVIEWER_ID_CANDIDATES = ["reviewer_id", "reviewer", "user_id", "author_id"]
LISTING_ID_CANDIDATES = ["listing_id", "listingId", "listing_id_str"]

def _pick_col(df: pd.DataFrame, candidates: list[str]) -> str | None:
    for c in candidates:
        if c in df.columns:
            return c
    return None

def _fmt_int_for_latex(n: int) -> str:
    # Render 1,533,422 as 1{,}533{,}422 for LaTeX
    return f"{int(n):,}".replace(",", "{,}")

def _fmt_pct(p: float) -> str:
    return f"{p*100:.2f}\\%"

def _date_span_for_latex(series: pd.Series) -> str:
    s = pd.to_datetime(series, errors="coerce")
    s = s.dropna()
    if s.empty:
        return "N/A"
    lo, hi = s.min(), s.max()
    # e.g., "Oct~2022--Aug~2025"
    return f"{lo.strftime('%b~%Y')}--{hi.strftime('%b~%Y')}"

def build_integrity_table(reviews_df: pd.DataFrame, listings_df: pd.DataFrame | None = None,
                          runtime_seconds: float | None = None) -> tuple[pd.DataFrame, str]:
    # Total reviews (cleaned corpus size)
    total_reviews = len(reviews_df)

    # Predictions generated (both deception & sentiment present and non-null)
    has_deception = reviews_df[PRED_COLS["deception_pred"][0]].notna() if PRED_COLS["deception_pred"][0] in reviews_df else pd.Series(False, index=reviews_df.index)
    has_sentiment = reviews_df[PRED_COLS["sent_pred"][0]].notna() if PRED_COLS["sent_pred"][0] in reviews_df else pd.Series(False, index=reviews_df.index)
    has_preds = (has_deception & has_sentiment)
    predictions_generated = int(has_preds.sum())
    coverage = predictions_generated / total_reviews if total_reviews else np.nan

    # Date span (from reviews.date)
    date_span = _date_span_for_latex(reviews_df.get("date", pd.Series(dtype=object)))

    # Unique listings
    listing_col = _pick_col(reviews_df, LISTING_ID_CANDIDATES)
    unique_listings_from_reviews = reviews_df[listing_col].nunique() if listing_col else np.nan
    if listings_df is not None and "id" in listings_df.columns:
        unique_listings = listings_df["id"].nunique()
    else:
        unique_listings = unique_listings_from_reviews

    # Unique reviewers
    rid_col = _pick_col(reviews_df, REVIEWER_ID_CANDIDATES)
    unique_reviewers = reviews_df[rid_col].nunique() if rid_col else np.nan

    # Throughput (rows/s), if runtime provided
    if runtime_seconds and runtime_seconds > 0:
        throughput = predictions_generated / float(runtime_seconds)
    else:
        throughput = np.nan

    # Build a tidy DataFrame (Metric, Description, Value)
    rows = [
        ("Total reviews processed", "Reviews in the cleaned Airbnb corpus", _fmt_int_for_latex(total_reviews)),
        ("Predictions generated", "Reviews with valid DistilBERT outputs", _fmt_int_for_latex(predictions_generated)),
        ("Prediction coverage", "Share of total reviews with predictions", _fmt_pct(coverage) if np.isfinite(coverage) else "N/A"),
        ("Date span", "Review period after preprocessing", date_span),
        ("Unique listings", "Distinct listings represented", _fmt_int_for_latex(unique_listings) if pd.notna(unique_listings) else "N/A"),
        ("Unique reviewers", "Distinct reviewer identifiers", _fmt_int_for_latex(unique_reviewers) if pd.notna(unique_reviewers) else "(insert count)"),
        ("Runtime (s)", "Total inference execution time", f"{runtime_seconds:.0f}" if runtime_seconds else "(insert value)"),
        ("Processing throughput", "Reviews processed per second", f"{throughput:.2f}" if pd.notna(throughput) else "(insert value)"),
    ]
    df = pd.DataFrame(rows, columns=["Metric", "Description", "Value"])

    # Emit LaTeX (booktabs) with your exact caption/label.
    # Note: Escaping is already handled in values; descriptions are plain text.
    latex_rows = "\n".join([f"{m} & {d} & {v} \\\\" for m, d, v in rows])
    latex_table = rf"""
\begin{{table}}[H]
\centering
\caption{{Prediction Coverage and Integrity Summary for Airbnb New Zealand Reviews.}}
\label{{tab:airbnb_inference_integrity}}
\begin{{tabular}}{{lcc}}
\toprule
\textbf{{Metric}} & \textbf{{Description}} & \textbf{{Value}} \\
\midrule
{latex_rows}
\bottomrule
\end{{tabular}}
\end{{table}}
""".strip()

    return df, latex_table

In [11]:
# RQ1_temporal_analysis.py
# End-to-end workflow for RQ1: temporal dynamics of deception & sentiment

import os
import math
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -----------------------------
# Config
# -----------------------------
INPUT_PATH = "outputs/airbnb_predictions.parquet"  # <- change if needed
OUT_DIR = "outputs/rq1_temporal"
ROLLING_WINDOW = 3  # months; set to 6 for 6-month smoothing
DATE_COL = "date"

# -----------------------------
# Utils
# -----------------------------
def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

def _has(df, cols):
    return all(c in df.columns for c in cols)

def _maybe_to_datetime(df, col):
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")
    return df

def _percentify(series_float_0_1):
    return series_float_0_1 * 100.0

def safe_read_parquet(path):
    try:
        return pd.read_parquet(path)
    except Exception as e:
        raise RuntimeError(f"Failed to read parquet at {path}: {e}")

# -----------------------------
# Load & prepare
# -----------------------------
def load_predictions(path: str) -> pd.DataFrame:
    df = safe_read_parquet(path)
    df = _maybe_to_datetime(df, DATE_COL)
    if df[DATE_COL].isna().all():
        raise ValueError(f"All {DATE_COL} values are NaT after parsing. Check your input.")
    return df

# -----------------------------
# Monthly aggregation
# -----------------------------
def compute_monthly(df: pd.DataFrame, rolling_window: int = 3) -> tuple[pd.DataFrame, pd.DataFrame]:
    # Ensure probabilities/labels exist; be robust to naming
    # Deception prob
    if "deception_prob_deceptive" not in df.columns:
        # try common fallback names
        candidates = [c for c in df.columns if c.startswith("deception_prob_")]
        if len(candidates) >= 2 and "deception_prob_deceptive" not in candidates:
            # best effort: pick the column containing 'decept' as P(deceptive)
            guess = [c for c in candidates if "decept" in c]
            if guess:
                df["deception_prob_deceptive"] = df[guess[0]]
            else:
                raise KeyError("Could not find deception probability columns.")
        else:
            raise KeyError("Missing 'deception_prob_deceptive' in predictions.")

    # Sentiment probabilities (neg/neu/pos). If not present, derive from sent_pred distribution
    have_sent_probs = _has(df, ["sent_prob_negative", "sent_prob_neutral", "sent_prob_positive"])
    if not have_sent_probs and "sent_pred" in df.columns:
        # derive dummy probs from labels (hard 1.0 for the predicted class)
        # assuming classes: 0=neg, 1=neu, 2=pos
        for k in ["sent_prob_negative", "sent_prob_neutral", "sent_prob_positive"]:
            if k not in df.columns:
                df[k] = 0.0
        df.loc[df["sent_pred"] == 0, "sent_prob_negative"] = 1.0
        df.loc[df["sent_pred"] == 1, "sent_prob_neutral"]  = 1.0
        df.loc[df["sent_pred"] == 2, "sent_prob_positive"] = 1.0
        have_sent_probs = True

    # Sentiment positive share from labels (more stable for % positive)
    def _pct_pos_from_pred(x: pd.Series):
        # assume 2 = positive (your DistilBERT training)
        return (x == 2).mean() if x.notna().any() else np.nan

    # Year-month key
    ym = pd.to_datetime(df[DATE_COL]).dt.to_period("M")
    df = df.copy()
    df["year_month"] = ym.astype(str)  # keep as string for csv friendliness
    # Aggregate monthly
    agg_dict = {
        "deception_prob_deceptive": "mean",
    }
    if have_sent_probs:
        agg_dict.update({
            "sent_prob_positive": "mean",
            "sent_prob_negative": "mean",
            "sent_prob_neutral":  "mean",
        })
    if "deception_pred" in df.columns:
        agg_dict["deception_pred"] = "mean"
    if "sent_pred" in df.columns:
        agg_dict["sent_pred"] = _pct_pos_from_pred

    monthly = df.groupby("year_month").agg(agg_dict).reset_index()

    # Sort by time
    monthly["year_month_dt"] = pd.to_datetime(monthly["year_month"])
    monthly = monthly.sort_values("year_month_dt")

    # Rolling averages
    roll_cols = [c for c in monthly.columns if c not in ["year_month", "year_month_dt"]]
    monthly_roll = monthly.copy()
    for c in roll_cols:
        monthly_roll[c] = monthly_roll[c].rolling(rolling_window, min_periods=1).mean()

    return monthly, monthly_roll

# -----------------------------
# Plots
# -----------------------------
def fig1_temporal_trends(monthly: pd.DataFrame, out_path: str):
    plt.figure(figsize=(10, 5))
    x = monthly["year_month_dt"]

    # Deception mean prob (as %)
    if "deception_prob_deceptive" in monthly.columns:
        y_dec = _percentify(monthly["deception_prob_deceptive"])
        plt.plot(x, y_dec, label="Mean P(deceptive) %")

    # Positive/negative sentiment (as %)
    if "sent_prob_positive" in monthly.columns:
        y_pos = _percentify(monthly["sent_prob_positive"])
        plt.plot(x, y_pos, label="Mean P(positive) %")

    if "sent_prob_negative" in monthly.columns:
        y_neg = _percentify(monthly["sent_prob_negative"])
        plt.plot(x, y_neg, label="Mean P(negative) %")

    plt.title("Monthly Trends: Deception & Sentiment Probabilities")
    plt.xlabel("Month")
    plt.ylabel("Probability (%)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_path, dpi=200)
    plt.close()

def fig2_dualaxis_deception_vs_positive(monthly: pd.DataFrame, out_path: str):
    # Single figure (one plot), but show two series with a second y-axis
    # (Matplotlib still counts this as one chart.)
    fig, ax1 = plt.subplots(figsize=(10, 5))
    x = monthly["year_month_dt"]
    y1 = monthly.get("deception_prob_deceptive", pd.Series(index=monthly.index, dtype=float))
    y2 = monthly.get("sent_prob_positive", pd.Series(index=monthly.index, dtype=float))

    ax1.plot(x, _percentify(y1), label="P(deceptive) %")
    ax1.set_xlabel("Month")
    ax1.set_ylabel("P(deceptive) (%)")

    ax2 = ax1.twinx()
    ax2.plot(x, _percentify(y2), label="P(positive) %")
    ax2.set_ylabel("P(positive) (%)")

    fig.suptitle("Dual-Axis Trend: Deception vs. Positive Sentiment")
    fig.tight_layout()
    fig.savefig(out_path, dpi=200)
    plt.close(fig)

def fig3_heatmap_month_year(monthly: pd.DataFrame, value_col: str, out_path: str):
    # Build Year x Month matrix for a chosen value (e.g., deception_prob_deceptive)
    temp = monthly.copy()
    temp["Year"] = temp["year_month_dt"].dt.year
    temp["Month"] = temp["year_month_dt"].dt.month
    pivot = temp.pivot(index="Year", columns="Month", values=value_col)

    # Plot with imshow
    plt.figure(figsize=(10, 5))
    im = plt.imshow(pivot, aspect="auto", interpolation="nearest")
    plt.title(f"Heatmap (Year x Month): {value_col}")
    plt.xlabel("Month")
    plt.ylabel("Year")

    # ticks
    plt.xticks(ticks=np.arange(1, 13) - 1, labels=list(range(1, 13)))
    plt.yticks(ticks=np.arange(len(pivot.index)), labels=pivot.index)

    # colorbar
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.tight_layout()
    plt.savefig(out_path, dpi=200)
    plt.close()

# -----------------------------
# Correlation & optional extras
# -----------------------------
def compute_correlations(monthly: pd.DataFrame) -> pd.DataFrame:
    cols = []
    if "deception_prob_deceptive" in monthly.columns:
        cols.append("deception_prob_deceptive")
    if "sent_prob_positive" in monthly.columns:
        cols.append("sent_prob_positive")
    if not cols:
        return pd.DataFrame()

    corr_pearson = monthly[cols].corr(method="pearson")
    corr_spearman = monthly[cols].corr(method="spearman")

    # Stack them with identifiers
    out = []
    for mname, cmat in [("pearson", corr_pearson), ("spearman", corr_spearman)]:
        c = cmat.reset_index().melt(id_vars="index", var_name="var2", value_name="corr")
        c = c.rename(columns={"index": "var1"})
        c["method"] = mname
        out.append(c)
    return pd.concat(out, ignore_index=True)

def optional_seasonal_decompose(monthly: pd.DataFrame, value_col: str, out_path: str):
    try:
        from statsmodels.tsa.seasonal import seasonal_decompose
    except Exception:
        warnings.warn("statsmodels not installed; skipping seasonal decomposition.")
        return

    s = monthly.set_index("year_month_dt")[value_col].dropna()
    # Ensure proper freq; infer monthly
    s = s.asfreq("MS")
    try:
        res = seasonal_decompose(s, model="additive", period=12)
    except Exception as e:
        warnings.warn(f"Decompose failed: {e}")
        return

    # Plot each component in a single figure (still one chart)
    plt.figure(figsize=(10, 8))
    ax = plt.subplot(4, 1, 1); ax.plot(res.observed); ax.set_title("Observed")
    ax = plt.subplot(4, 1, 2); ax.plot(res.trend);    ax.set_title("Trend")
    ax = plt.subplot(4, 1, 3); ax.plot(res.seasonal); ax.set_title("Seasonal")
    ax = plt.subplot(4, 1, 4); ax.plot(res.resid);    ax.set_title("Residual")
    plt.tight_layout()
    plt.savefig(out_path, dpi=200)
    plt.close()

def optional_change_points(monthly: pd.DataFrame, value_col: str, out_path: str, n_bkps: int = 3):
    try:
        import ruptures as rpt
    except Exception:
        warnings.warn("ruptures not installed; skipping change-point detection.")
        return

    y = monthly[value_col].dropna().to_numpy().reshape(-1, 1)
    if len(y) < (n_bkps + 2):
        warnings.warn("Not enough points for change-point detection; skipping.")
        return

    algo = rpt.Pelt(model="rbf").fit(y)
    bks = algo.predict(pen=10)  # heuristic; adjust if needed

    # Plot signal with breakpoints
    plt.figure(figsize=(10, 4))
    plt.plot(monthly["year_month_dt"], y)
    for b in bks[:-1]:
        # b is index in the 1..N domain; align to x
        if 0 <= b-1 < len(monthly):
            xval = monthly["year_month_dt"].iloc[b-1]
            plt.axvline(xval)
    plt.title(f"Change-Point Detection: {value_col}")
    plt.xlabel("Month")
    plt.ylabel(value_col)
    plt.tight_layout()
    plt.savefig(out_path, dpi=200)
    plt.close()

# -----------------------------
# Main
# -----------------------------
def main():
    ensure_dir(OUT_DIR)
    df = load_predictions(INPUT_PATH)

    monthly, monthly_roll = compute_monthly(df, rolling_window=ROLLING_WINDOW)

    # Save tables
    monthly.to_csv(os.path.join(OUT_DIR, "monthly_trends.csv"), index=False)
    monthly_roll.to_csv(os.path.join(OUT_DIR, "monthly_trends_rolling.csv"), index=False)

    # Figures
    fig1_temporal_trends(monthly, os.path.join(OUT_DIR, "fig1_temporal_trends.png"))
    fig2_dualaxis_deception_vs_positive(monthly, os.path.join(OUT_DIR, "fig2_dualaxis_deception_vs_positive.png"))
    # Heatmap on deception by default (you can switch value_col to "sent_prob_positive")
    fig3_heatmap_month_year(monthly, "deception_prob_deceptive", os.path.join(OUT_DIR, "fig3_heatmap_month_year.png"))

    # Correlations
    corr = compute_correlations(monthly)
    if not corr.empty:
        corr.to_csv(os.path.join(OUT_DIR, "correlations.csv"), index=False)

    # Optional extras (run only if libraries installed)
    optional_seasonal_decompose(monthly, "deception_prob_deceptive", os.path.join(OUT_DIR, "fig4_seasonal_decompose_deception.png"))
    optional_change_points(monthly, "deception_prob_deceptive", os.path.join(OUT_DIR, "fig5_change_points.png"))

    print(f"Done. Outputs saved in: {OUT_DIR}")

if __name__ == "__main__":
    main()

Done. Outputs saved in: outputs/rq1_temporal


/tmp/ipython-input-1957896985.py:250: UserWarning: ruptures not installed; skipping change-point detection.
  warnings.warn("ruptures not installed; skipping change-point detection.")


In [ ]:
!python run_airbnb_inference.py \
  --input_parquet data/airbnb_reviews_cleaned.parquet \
  --output_parquet outputs/airbnb_predictions.parquet \
  --deception_model_dir "/content/drive/MyDrive/Colab Notebooks/airbnb_nz_deception_sentiment/artifacts/distilbert_deception_sweep/best_model" \
  --sentiment_model_dir "/content/drive/MyDrive/Colab Notebooks/airbnb_nz_deception_sentiment/artifacts/distilbert_sentiment_sweep/best_model"

In [ ]:
!python run_airbnb_inference.py \
  --input_parquet data/airbnb_reviews_cleaned.parquet \
  --output_parquet outputs/airbnb_predictions.parquet \
  --deception_model_dir "/content/drive/MyDrive/Colab Notebooks/airbnb_nz_deception_sentiment/artifacts/deception_distilbert_hf" \
  --sentiment_model_dir "models/distilbert_tripadvisor"


2025-10-22 01:02:27.859102: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761094947.923640   14048 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761094947.948451   14048 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761094947.998164   14048 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761094947.998199   14048 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761094947.998207   14048 computation_placer.cc:177] computation placer alr